In [ ]:
# Installations
!pip install datasets==1.0.2
!pip install transformers
!pip install rouge_score
!pip install urduhack

     |████████████████████████████████| 1.8MB 6.6MB/s 
     |████████████████████████████████| 245kB 25.1MB/s 
     |████████████████████████████████| 2.5MB 6.8MB/s 
     |████████████████████████████████| 3.3MB 33.5MB/s 
     |████████████████████████████████| 901kB 34.4MB/s 
     |████████████████████████████████| 112kB 7.1MB/s 
     |████████████████████████████████| 3.4MB 8.8MB/s 
     |████████████████████████████████| 686kB 34.8MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


In [ ]:
import datasets
import transformers
import pandas as pd
from datasets import Dataset
#preprocessing
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import remove_accents
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import normalize_whitespace
#Tokenizer
from transformers import RobertaTokenizerFast

#Encoder-Decoder Model
from transformers import EncoderDecoderModel

#Training
from transformers import Seq2SeqTrainer
from transformers import TrainingArguments
from dataclasses import dataclass, field
from typing import Optional

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")
#model = AutoModelForMaskedLM.from_pretrained("urduhack/roberta-urdu-small")

In [ ]:
data=pd.read_csv("/content/drive/MyDrive/Urdu Text Summarization/BBC.csv")
data.drop(columns=['Topics'],axis=1,inplace=True)
data = data.dropna()
print("Data size:",len(data))
data.head()

FileNotFoundError: ignored

In [ ]:
def cleaning(data,col_name):
  lst=[]
  i=0;
  for x in data[col_name]:
    output = remove_punctuation(str(x))
    output=remove_accents(output)
    output=replace_numbers(output)
    output = normalize_whitespace(output)
    lst.append(output)
  return lst

data["Summary"]=cleaning(data,"Summary")
data["Text"]=cleaning(data,"Text")

In [ ]:
train_data=Dataset.from_pandas(data[:380])
val_data=Dataset.from_pandas(data[300:400])
test_data=Dataset.from_pandas(data[380:475])

In [ ]:
#tokenizer = RobertaTokenizerFast.from_pretrained("urduhack/roberta-urdu-small")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
#parameter setting
batch_size=10  #
encoder_max_length=300
decoder_max_length=30

def process_data_to_model_inputs(batch):
  # tokenize the inputs and labels
  inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(batch["Summary"], padding="max_length", truncation=True, max_length=decoder_max_length)

  batch["input_ids"] = inputs.input_ids
  batch["attention_mask"] = inputs.attention_mask
  batch["decoder_input_ids"] = outputs.input_ids
  batch["decoder_attention_mask"] = outputs.attention_mask
  batch["labels"] = outputs.input_ids.copy()

  # because RoBERTa automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`. 
  # We have to make sure that the PAD token is ignored
  batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in batch["labels"]]

  return batch

#processing training data
train_data = train_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"]
)
train_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

#processing validation data
val_data = val_data.map(
    process_data_to_model_inputs, 
    batched=True, 
    batch_size=batch_size, 
    remove_columns=["Text", "Summary"]
)
val_data.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)


In [ ]:
from transformers import EncoderDecoderModel
roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained("urduhack/roberta-urdu-small", "urduhack/roberta-urdu-small", tie_encoder_decoder=True)

Some weights of the model checkpoint at urduhack/roberta-urdu-small were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForCausalLM were not initialized from the model checkpoint at urduhack/roberta-urdu-small and are newly initialized: ['roberta.encoder.layer.7.crossattention.self.key.weight', 'roberta.encoder.layer.2.crossattention.output.dense.bias',

In [ ]:
# set special tokens
roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id                                             
roberta_shared.config.eos_token_id = tokenizer.eos_token_id

# sensible parameters for beam search
# set decoding params                               
roberta_shared.config.max_length = 30
roberta_shared.config.early_stopping = True
roberta_shared.config.no_repeat_ngram_size = 1
roberta_shared.config.length_penalty = 2.0
roberta_shared.config.num_beams = 4
roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

In [ ]:
# load rouge for validation
rouge = datasets.load_metric("rouge")
def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
from transformers import Seq2SeqTrainingArguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Urdu Text Summarization",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    predict_with_generate=True,
#    evaluate_during_training=True,
    do_train=True,
    do_eval=True,
    logging_steps=2, 
    save_steps=3, 
    eval_steps=10, 
    warmup_steps=10, 
    overwrite_output_dir=True,
    save_total_limit=1,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=roberta_shared,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=val_data,
)
trainer.train()

The following columns in the training set  don't have a corresponding argument in `EncoderDecoderModel.forward` and have been ignored: __index_level_0__.
***** Running training *****
  Num examples = 380
  Num Epochs = 3
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 1
  Total optimization steps = 114


Step,Training Loss
2,11.912000
4,11.005100
6,8.394800
8,7.568800
10,7.505800
12,7.270700
14,7.270800
16,7.071700
18,6.769800
20,6.586000


Saving model checkpoint to /content/drive/MyDrive/Urdu Text Summarization/checkpoint-3
Configuration saved in /content/drive/MyDrive/Urdu Text Summarization/checkpoint-3/config.json
Model weights saved in /content/drive/MyDrive/Urdu Text Summarization/checkpoint-3/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/Urdu Text Summarization/checkpoint-6
Configuration saved in /content/drive/MyDrive/Urdu Text Summarization/checkpoint-6/config.json
Model weights saved in /content/drive/MyDrive/Urdu Text Summarization/checkpoint-6/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/Urdu Text Summarization/checkpoint-3] due to args.save_total_limit
Saving model checkpoint to /content/drive/MyDrive/Urdu Text Summarization/checkpoint-9
Configuration saved in /content/drive/MyDrive/Urdu Text Summarization/checkpoint-9/config.json
Model weights saved in /content/drive/MyDrive/Urdu Text Summarization/checkpoint-9/pytorch_model.bin
Deleting older checkpoint [/conten

TrainOutput(global_step=114, training_loss=5.687734779558684, metrics={'train_runtime': 5507.9085, 'train_samples_per_second': 0.207, 'train_steps_per_second': 0.021, 'total_flos': 349841369433600.0, 'train_loss': 5.687734779558684, 'epoch': 3.0})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("urduhack/roberta-urdu-small")
model = EncoderDecoderModel.from_pretrained("/content/drive/MyDrive/checkpoint-114")
model
batch_size = 20

# map data correctly
def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    inputs = tokenizer(batch["Text"], padding="max_length", truncation=True, max_length=5, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    batch["pred"] = output_str
    return batch
results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["Text"])
pred_str = results["pred"]
label_str = results["Summary"]

The following encoder weights were not tied to the decoder ['roberta/pooler']
The following encoder weights were not tied to the decoder ['roberta/pooler']
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
print("ROUGE 1 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid)
print("ROUGE 2 SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid)
print("ROUGE L SCORE: ",rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rougeL"])["rougeL"].mid)

ROUGE 1 SCORE:  Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE 2 SCORE:  Score(precision=0.0, recall=0.0, fmeasure=0.0)
ROUGE L SCORE:  Score(precision=0.0, recall=0.0, fmeasure=0.0)


In [ ]:
test_data["Text"][0]

'انگریزی چینل کے دونوں اطراف میں خوشی کی آواز بھڑک اٹھی جب چینل سرنگ کھودنے والے برطانوی اور فرانسیسی کارکن آخر کار اس راستے پر دستک دینے کے بعد ملے جس سے چلنے اور ہاتھ ملایا جاتا تھا فرانسیسی صدر فرانسوا میترراینڈ نے کہا کہ آج پہلی بار مرد زیر زمین چینل کو عبور کرسکتے ہیں ہمارے دونوں ممالک کی جیورنبل کی کتنی روشن علامت ہے پیش رفت فٹ لمبی خدمت کی سرنگ میں آئی ہے جو دو ریل سرنگوں کو برقرار رکھنے کے لئے استعمال ہوگی جو اب بھی بور ہیں اس نے یورپ کے سب سے بڑے انجینئرنگ پروجیکٹ میں ایک علامتی سنگ میل کا نشان لگایا ڈوور انگلینڈ کے جیک ہیمرز گراہم فگ اور کلیس فرانس کے فیلیپ کوزٹ نے سرنگ کے برطانوی اور فرانسیسی اطراف کو جوڑنے کے لئے چاک کا آخری پاؤنڈ کھٹکھٹایا جس کو چنبل کا نام دیا گیا ہے مسکراتے ہوئے جوڑے نے پھر ہاتھ جوڑ کر اپنے قومی جھنڈوں کا تبادلہ کیا مجموعی طور پر کارکنوں نے ان کو دیکھا اور ان کی تعریف کی خدا کو بچانے والی ملکہ نے فرانسیسی کارکنوں کو شیمپین کی بوتلوں سے پردہ اٹھاتے ہوئے رویا وائیو لا فرانس کا جواب برطانوی کی طرف سے آیا ہفتے کے روز ہینڈ شیک تین دن تک آئے تھے جس دن کلیس کے 

In [ ]:
pred_str

['پاکستان کے پولیس افسر نے کہنا ہے کہ پاکستان کو پرواز کرنے کی کوشش کر رہے ہیں لیکن',
 'پاکستان کے پولیس افسر نے بتایا ہے کہ وہ کو گولی مار لیا گیا تھا جب اسے اپنے گھر میں جا لے جائے اور جہاز سے',
 'پاکستان اور زمبابوے کے خلاف ٹیسٹ سیریز میں شکست دے دیا گیا ہے کہ پاکستان ٹیم کو',
 'پاکستان کے حکام نے کہا ہے کہ پاکستان اور فوج کی مدد کرنے والے شہریوں کو',
 'پاکستان اور زمبابوے کے خلاف ٹیسٹ سیریز میں ایک میچ کی فتح کر دی گئی جس کو',
 'پاکستان اور زمبابوے کے درمیان کھیلے جانے والے میچ سے دو رنز کی فتح ہو گیا جب پاکستان کو',
 'پاکستان اور زمبابوے کے درمیان ٹیسٹ سیریز میں دو رنز کی برتری سے شکست دی گئی تھی تاہم',
 'پاکستان اور زمبابوے کی ٹیموں کے درمیان دو ٹیسٹ میچوں میں ایک روزہ میچ کا مقابلہ کیا گیا جس نے',
 'پاکستان اور زمبابوے کے درمیان ٹیسٹ سیریز میں ایک میچ کا نقصان ہو گیا تھا جب اس نے',
 'پاکستان اور زمبابوے کے درمیان کھیلے جانے والے میچ میں دو رنز کی اننگز سے کھیل آئے تھے لیکن ایک وکٹ کا نقصان نہیں ہوتا تھا',
 'پاکستان کرکٹ بورڈ نے ایک انٹرویو میں کہا کہ پاکستان کے خلاف سیریز کی سزا